In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, f_oneway
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*80)
print("ALPHACARE INSURANCE SOLUTIONS - COMPLETE ANALYSIS PIPELINE")
print("="*80)

ALPHACARE INSURANCE SOLUTIONS - COMPLETE ANALYSIS PIPELINE


In [9]:
# Peek at first few lines
with open('./Data/MachineLearningRating_v3.txt', 'r') as f:
    for i, line in enumerate(f):
        if i < 5:  # Show first 5 lines
            print(repr(line))  # repr() shows hidden characters like tabs
        else:
            break

'UnderwrittenCoverID|PolicyID|TransactionMonth|IsVATRegistered|Citizenship|LegalType|Title|Language|Bank|AccountType|MaritalStatus|Gender|Country|Province|PostalCode|MainCrestaZone|SubCrestaZone|ItemType|mmcode|VehicleType|RegistrationYear|make|Model|Cylinders|cubiccapacity|kilowatts|bodytype|NumberOfDoors|VehicleIntroDate|CustomValueEstimate|AlarmImmobiliser|TrackingDevice|CapitalOutstanding|NewVehicle|WrittenOff|Rebuilt|Converted|CrossBorder|NumberOfVehiclesInFleet|SumInsured|TermFrequency|CalculatedPremiumPerTerm|ExcessSelected|CoverCategory|CoverType|CoverGroup|Section|Product|StatutoryClass|StatutoryRiskType|TotalPremium|TotalClaims\n'
'145249|12827|2015-03-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not specified|Not specified|South Africa|Gauteng|1459|Rand East|Rand East|Mobility - Motor|44069150|Passenger Vehicle|2004|MERCEDES-BENZ|E 240|6|2597|130|S/D|4|6/2002|119300|Yes|No|119300|More than 6 months||||||0.01|Monthly|25|Mobility - Winds

In [14]:
# ============================================================================
# TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING
# ============================================================================

print("\n" + "="*80)
print("TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING")
print("="*80)

# Load the data
print("\n[1.1] Loading data...")
df = pd.read_csv('./Data/MachineLearningRating_v3.txt', sep='|') 



print("Data loaded successfully!")
print(f"Dataset shape: {df.shape}")

print("\n[1.2] Data Understanding - Basic Information")
print("-" * 80)
print(df.info())
print("\nFirst few rows:")
print(df.head())


TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING

[1.1] Loading data...
Data loaded successfully!
Dataset shape: (1000098, 52)

[1.2] Data Understanding - Basic Information
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   objec

In [16]:
print("\n[1.3] Data Quality Assessment")
print("-" * 80)
# Function to assess data quality
def assess_data_quality(df):
    """Comprehensive data quality assessment"""
    quality_report = pd.DataFrame({
        'Column': df.columns,
        'Data_Type': df.dtypes,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
        'Unique_Values': df.nunique(),
        'Sample_Values': [df[col].dropna().unique()[:3].tolist() if len(df[col].dropna().unique()) > 0 else [] for col in df.columns]
    })
    
    print("Data Quality Report:")
    print(quality_report[quality_report.Missing_Count > 0].sort_values('Missing_Percentage', ascending=False))
    
    return quality_report
quality_report = assess_data_quality(df)


[1.3] Data Quality Assessment
--------------------------------------------------------------------------------
Data Quality Report:
                                          Column Data_Type  Missing_Count  \
NumberOfVehiclesInFleet  NumberOfVehiclesInFleet   float64        1000098   
CrossBorder                          CrossBorder    object         999400   
CustomValueEstimate          CustomValueEstimate   float64         779642   
WrittenOff                            WrittenOff    object         641901   
Converted                              Converted    object         641901   
Rebuilt                                  Rebuilt    object         641901   
NewVehicle                            NewVehicle    object         153295   
Bank                                        Bank    object         145961   
AccountType                          AccountType    object          40232   
Gender                                    Gender    object           9536   
MaritalStatus       

In [17]:
print("DataFrame shape:", df.shape)
print("Columns:", list(df.columns))
print("\nTotal missing values per column:")
print(df.isnull().sum())
print("\nAny missing at all?", df.isnull().values.any())

DataFrame shape: (1000098, 52)
Columns: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']

Total missing values per column:
UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered 

In [18]:
print("\n[1.4] Descriptive Statistics")
print("-" * 80)

def calculate_descriptive_stats(df):
    """Calculate comprehensive descriptive statistics"""
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    stats_summary = df[numerical_cols].describe().T
    stats_summary['variance'] = df[numerical_cols].var()
    stats_summary['skewness'] = df[numerical_cols].skew()
    stats_summary['kurtosis'] = df[numerical_cols].kurtosis()
    
    print("Descriptive Statistics for Numerical Variables:")
    print(stats_summary)
    
    return stats_summary
stats_summary = calculate_descriptive_stats(df)


[1.4] Descriptive Statistics
--------------------------------------------------------------------------------
Descriptive Statistics for Numerical Variables:
                              count          mean           std           min  \
UnderwrittenCoverID       1000098.0  1.048175e+05  6.329371e+04  1.000000e+00   
PolicyID                  1000098.0  7.956682e+03  5.290039e+03  1.400000e+01   
PostalCode                1000098.0  3.020601e+03  2.649854e+03  1.000000e+00   
mmcode                     999546.0  5.487770e+07  1.360381e+07  4.041200e+06   
RegistrationYear          1000098.0  2.010225e+03  3.261391e+00  1.987000e+03   
Cylinders                  999546.0  4.046642e+00  2.940201e-01  0.000000e+00   
cubiccapacity              999546.0  2.466743e+03  4.428006e+02  0.000000e+00   
kilowatts                  999546.0  9.720792e+01  1.939326e+01  0.000000e+00   
NumberOfDoors              999546.0  4.019250e+00  4.683144e-01  0.000000e+00   
CustomValueEstimate        2204

In [19]:
print("\n[1.5] Key Business Metrics Calculation")
print("-" * 80)

def calculate_business_metrics(df):
    """Calculate key insurance business metrics"""
    
    # Overall metrics
    overall_metrics = {
        'Total_Premium': df['TotalPremium'].sum(),
        'Total_Claims': df['TotalClaims'].sum(),
        'Overall_Loss_Ratio': df['TotalClaims'].sum() / df['TotalPremium'].sum(),
        'Number_of_Policies': df['PolicyID'].nunique(),
        'Claim_Frequency': (df['TotalClaims'] > 0).mean(),
        'Average_Premium': df['TotalPremium'].mean(),
        'Average_Claim': df[df['TotalClaims'] > 0]['TotalClaims'].mean(),
        'Total_Margin': (df['TotalPremium'] - df['TotalClaims']).sum()
    }
    
    print("Overall Business Metrics:")
    for metric, value in overall_metrics.items():
        if 'Ratio' in metric or 'Frequency' in metric:
            print(f"{metric}: {value:.2%}")
        else:
            print(f"{metric}: {value:,.2f}")
    
    # Loss ratio by Province
    print("\n\nLoss Ratio by Province:")
    loss_by_province = df.groupby('Province').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_province['LossRatio'] = loss_by_province['TotalClaims'] / loss_by_province['TotalPremium']
    loss_by_province = loss_by_province.sort_values('LossRatio', ascending=False)
    print(loss_by_province)
    
    # Loss ratio by VehicleType
    print("\n\nLoss Ratio by Vehicle Type:")
    loss_by_vehicle = df.groupby('VehicleType').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_vehicle['LossRatio'] = loss_by_vehicle['TotalClaims'] / loss_by_vehicle['TotalPremium']
    loss_by_vehicle = loss_by_vehicle.sort_values('LossRatio', ascending=False)
    print(loss_by_vehicle)
    
    # Loss ratio by Gender
    print("\n\nLoss Ratio by Gender:")
    loss_by_gender = df.groupby('Gender').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_gender['LossRatio'] = loss_by_gender['TotalClaims'] / loss_by_gender['TotalPremium']
    print(loss_by_gender)
    
    return overall_metrics, loss_by_province, loss_by_vehicle, loss_by_gender

overall_metrics, loss_by_province, loss_by_vehicle, loss_by_gender = calculate_business_metrics(df)



[1.5] Key Business Metrics Calculation
--------------------------------------------------------------------------------
Overall Business Metrics:
Total_Premium: 61,911,562.70
Total_Claims: 64,867,546.17
Overall_Loss_Ratio: 104.77%
Number_of_Policies: 7,000.00
Claim_Frequency: 0.28%
Average_Premium: 61.91
Average_Claim: 23,273.39
Total_Margin: -2,955,983.47


Loss Ratio by Province:
                TotalClaims  TotalPremium  PolicyID  LossRatio
Province                                                      
Gauteng        2.939415e+07  2.405377e+07    393865   1.222018
KwaZulu-Natal  1.430138e+07  1.320908e+07    169781   1.082693
Western Cape   1.038977e+07  9.806559e+06    170796   1.059472
North West     5.920250e+06  7.490508e+06    143287   0.790367
Mpumalanga     2.044675e+06  2.836292e+06     52718   0.720897
Free State     3.549223e+05  5.213632e+05      8099   0.680758
Limpopo        1.016477e+06  1.537324e+06     24836   0.661199
Eastern Cape   1.356427e+06  2.140104e+06     3

In [20]:
print("\n[1.6] Univariate Analysis - Distributions")
print("-" * 80)

def univariate_analysis(df):
    """Perform univariate analysis with visualizations"""
    
    numerical_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                      'CalculatedPremiumPerTerm', 'CustomValueEstimate']
    
    # Histograms
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, col in enumerate(numerical_cols):
        if col in df.columns:
            axes[idx].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('univariate_distributions.png', dpi=300, bbox_inches='tight')
    print("Saved: univariate_distributions.png")
    plt.close()
    
    # Box plots for outlier detection
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, col in enumerate(['TotalPremium', 'TotalClaims', 'CustomValueEstimate']):
        if col in df.columns:
            axes[idx].boxplot(df[col].dropna())
            axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
            axes[idx].set_ylabel(col)
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('boxplots_outliers.png', dpi=300, bbox_inches='tight')
    print("Saved: boxplots_outliers.png")
    plt.close()
    
    # Categorical variables
    categorical_cols = ['Province', 'VehicleType', 'Gender', 'CoverType']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, col in enumerate(categorical_cols):
        if col in df.columns:
            value_counts = df[col].value_counts().head(10)
            axes[idx].bar(range(len(value_counts)), value_counts.values)
            axes[idx].set_xticks(range(len(value_counts)))
            axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
            axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[idx].set_ylabel('Count')
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('categorical_distributions.png', dpi=300, bbox_inches='tight')
    print("Saved: categorical_distributions.png")
    plt.close()

univariate_analysis(df)


[1.6] Univariate Analysis - Distributions
--------------------------------------------------------------------------------
Saved: univariate_distributions.png
Saved: boxplots_outliers.png
Saved: categorical_distributions.png


In [ ]:
print("\n[1.7] Bivariate and Multivariate Analysis")
print("-" * 80)

def bivariate_analysis(df):
    """Perform bivariate and multivariate analysis"""
    
    # Correlation matrix
    numerical_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                      'CalculatedPremiumPerTerm', 'CustomValueEstimate']
    
    if all(col in df.columns for col in numerical_cols):
        corr_matrix = df[numerical_cols].corr()
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                    square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Correlation Matrix - Key Financial Variables', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
        print("Saved: correlation_matrix.png")
        plt.close()
    
    # Premium vs Claims by PostalCode
    postal_analysis = df.groupby('PostalCode').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum'
    }).reset_index()
    
    plt.figure(figsize=(14, 8))
    plt.scatter(postal_analysis['TotalPremium'], 
               postal_analysis['TotalClaims'], 
               alpha=0.6, s=100, edgecolors='black', linewidth=0.5)
    plt.xlabel('Total Premium', fontsize=12)
    plt.ylabel('Total Claims', fontsize=12)
    plt.title('Premium vs Claims by PostalCode', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Add diagonal line
    max_val = max(postal_analysis['TotalPremium'].max(), postal_analysis['TotalClaims'].max())
    plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Break-even line')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('premium_vs_claims_postal.png', dpi=300, bbox_inches='tight')
    print("Saved: premium_vs_claims_postal.png")
    plt.close()
    
    # Temporal trends
    df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
    df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)
    
    monthly_trends = df.groupby('TransactionMonth').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'HasClaim': 'mean'
    }).reset_index()
    
    fig, ax1 = plt.subplots(figsize=(16, 8))
    ax2 = ax1.twinx()
    
    ax1.plot(monthly_trends['TransactionMonth'], 
             monthly_trends['TotalClaims'], 
             'b-', linewidth=2, label='Total Claims', marker='o')
    ax2.plot(monthly_trends['TransactionMonth'], 
             monthly_trends['HasClaim'], 
             'r-', linewidth=2, label='Claim Frequency', marker='s')
    
    ax1.set_xlabel('Month', fontsize=12)
    ax1.set_ylabel('Total Claims', color='b', fontsize=12)
    ax2.set_ylabel('Claim Frequency', color='r', fontsize=12)
    ax1.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    
    plt.title('Temporal Trends in Claims and Frequency', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.tight_layout()
    plt.savefig('temporal_trends.png', dpi=300, bbox_inches='tight')
    print("Saved: temporal_trends.png")
    plt.close()
bivariate_analysis(df)